In [1]:
import os
from uuid import uuid4
from operator import itemgetter
from typing import Optional, List, Tuple

from langchain.chat_models import ChatOpenAI
from langchain.memory.chat_message_histories import ZepChatMessageHistory
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.prompts.prompt import PromptTemplate
from langchain.schema import AIMessage, HumanMessage
from langchain.schema.chat_history import BaseChatMessageHistory
from langchain.schema.runnable.history import RunnableWithMessageHistory
from langchain.vectorstores.zep import CollectionConfig, ZepVectorStore

from langchain_core.runnables import (
    ConfigurableField,
    RunnableBranch,
    RunnableLambda,
    RunnableParallel,
    RunnablePassthrough,
)
from langchain_core.runnables.utils import ConfigurableFieldSingleOption


In [2]:
ZEP_API_URL = os.environ.get("ZEP_API_URL", "http://localhost:8000")
ZEP_API_KEY = os.environ.get("ZEP_API_KEY", None)
ZEP_COLLECTION_NAME = os.environ.get("ZEP_COLLECTION", "langchaintest")

In [3]:
session_id = str(uuid4())
session_id

'86bb33b0-f5df-4f20-a7b0-7f70d2b8bd16'

In [3]:
collection_config = CollectionConfig(
    name=ZEP_COLLECTION_NAME,
    description="Zep collection for LangChain",
    metadata={},
    embedding_dimensions=1536,
    is_auto_embedded=True,
)

In [4]:
vectorstore = ZepVectorStore(
    collection_name=ZEP_COLLECTION_NAME,
    config=collection_config,
    api_url=ZEP_API_URL,
    api_key=ZEP_API_KEY,
    embedding=None,
)

In [ ]:
# Zep offers native, hardware-accelerated MMR. Enabling this will improve
# the diversity of results, but may also reduce relevance. You can tune
# the lambda parameter to control the tradeoff between relevance and diversity.
# Enabling is a good default.
retriever = vectorstore.as_retriever().configurable_fields(
    search_type=ConfigurableFieldSingleOption(
        id="search_type",
        options={"Similarity": "similarity", "Similarity with MMR Reranking": "mmr"},
        default="mmr",
        name="Search Type",
        description="Type of search to perform: 'similarity' or 'mmr'",
    ),
    search_kwargs=ConfigurableField(
        id="search_kwargs",
        name="Search kwargs",
        description=(
            "Specify 'k' for number of results to return and 'lambda_mult' for tuning"
            " MMR relevance vs diversity."
        ),
    ),
)

In [4]:

# chat = ChatOpenAI(
#     openai_api_key=OPENAI_API_KEY,
#     temperature=0,
#     model='gpt-3.5-turbo'
# )

In [4]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You're an assistant who's good at {ability}"),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{question}"),
    ]
)

chain = prompt | ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

### Adding Zep's message history

In [9]:
chain_with_history = RunnableWithMessageHistory(
    chain,
    lambda session_id: ZepChatMessageHistory(session_id, url=ZEP_API_URL),
    input_messages_key="question",
    history_messages_key="chat_history",
)

In [10]:
chain_with_history 

RunnableWithMessageHistory(bound=RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  chat_history: RunnableBinding(bound=RunnableLambda(...), config={'run_name': 'load_history'})
}), config={'run_name': 'insert_history'})
| RunnableBinding(bound=ChatPromptTemplate(input_variables=['ability', 'chat_history', 'question'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['ability'], template="You're an assistant who's good at {ability}")), MessagesPlaceholder(variable_name='chat_history'), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], template='{question}'))])
  | ChatOpenAI(client=<openai.resource

In [11]:
chain_with_history.invoke(
    {"ability": "math", "question": "What does cosine mean?"},
    config={"configurable": {"session_id": '86bb33b0-f5df-4f20-a7b0-7f70d2b8bd16'}},
)

AIMessage(content='Cosine is a mathematical function that is commonly used in trigonometry. In a right triangle, the cosine of an angle is defined as the ratio of the length of the adjacent side to the length of the hypotenuse. It is denoted as cos(θ), where θ represents the angle. Cosine can also be defined using the unit circle, where it gives the x-coordinate of a point on the circle corresponding to a given angle.')

In [13]:
chain_with_history.invoke(
    {"ability": "math", "question": "What's its inverse"},
    config={"configurable": {"session_id": '86bb33b0-f5df-4f20-a7b0-7f70d2b8bd16'}},
)

AIMessage(content='The inverse of the cosine function is called the arccosine or inverse cosine function. It is denoted as acos(x) or cos^(-1)(x). The arccosine function takes an input value between -1 and 1 and returns the angle whose cosine is equal to that input value. In other words, if y = cos^(-1)(x), then x = cos(y). The output of the arccosine function is typically given in radians, but it can also be expressed in degrees depending on the context.')